# Multi-species Hi-C data downloading

This notebook illustrates how to download the multi-species Hi-C file from DNAZoo data.

In [ ]:
# 1. Get the url list for downloading the hic files from dnazoo
import os
import sys
import json 
import requests
 
def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Finish download {filename}")
    else:
        print(f"Request fail: {response.status_code}")


input_txt = "../data/species_list.txt"
output_dir = "../data/dnazoo_json/"
os.makedirs(output_dir, exist_ok=True)
species_list = []
with open(input_txt) as f:
    for line in f:
        species_list.append(line.strip("\n"))
#https://dnazoo.s3.wasabisys.com/Acyrthosiphon_kondoi/README.json
final_url_list = []
root_url = "https://dnazoo.s3.wasabisys.com/"
miss_list=[]
for species in species_list:
    if "/" not in species:
        species += "/"
    readme_url = root_url + species + "README.json"

    readme_local = os.path.join(output_dir, species.replace("/","") + "_readme.json")
    if not os.path.exists(readme_local) or os.path.getsize(readme_local) <= 10:
        #os.system(f"wget {readme_url} -O {readme_local}")
        download_file(readme_url, readme_local)
    if not os.path.exists(readme_local) or os.path.getsize(readme_local) <= 10:
        print(f"Error: {readme_local} does not exist or is empty")
        miss_list.append(species)
        continue
    json_info = json.load(open(readme_local,"r"))
    hic_name = json_info['links']['cMap']['name']
    hic_url = root_url + species + hic_name
    if "hicUrl" in json_info['links']['cMap']:
        
        dropbox_url = json_info['links']['cMap']["hicUrl"]
        dropbox_url = dropbox_url.split("?")[0]
    else:
        dropbox_url = hic_url
    #final_url_list.append(hic_url)
    new_dict={species:[hic_url, dropbox_url]}
    final_url_list.append(new_dict)
import pickle
pickle.dump(final_url_list, open("../data/dnazoo_url_list.pkl","wb"))
print("Done")
print("Missed species:", miss_list)
#clean the json file directories
os.rmdir(output_dir)

In [ ]:
# 2. Download the hic files from the url list
import os
import sys
import pickle
import requests
 
def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Finish download {filename}")
    else:
        print(f"Request fail: {response.status_code}")
input_pkl = "../data/dnazoo_url_list.pkl"
output_dir ="../data/multi_species_hic/"
os.makedirs(output_dir, exist_ok=True)
data = pickle.load(open(input_pkl, 'rb'))
def download_url(url,file_path):
    os.system(f"wget {url} -O {file_path}")

from multiprocessing import Pool
#you can consider use the above choice to download multiple Hi-C simultaneously to accelerate downloading speed.
for species in data:
    url_list = data[species]
    file_path = os.path.join(output_dir, f"{species}.hic")
    if isinstance(url_list, list):
        for url in url_list:
            download_url(url, file_path)
            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                break
    elif isinstance(url_list, str):
        download_url(url_list, file_path)
    else:
        print(f"Error: {species} url list is not valid")
print("Done")



In [1]:
# 3. download all associated fasta files 
import os
import sys
import json 
import requests
 
def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Finish download {filename}")
    else:
        print(f"Request fail: {response.status_code}")
 

input_txt = "../data/species_list.txt"
output_dir = "../data/dnazoo_json/"
os.makedirs(output_dir, exist_ok=True)
species_list = []
with open(input_txt) as f:
    for line in f:
        species_list.append(line.strip("\n"))
#https://dnazoo.s3.wasabisys.com/Acyrthosiphon_kondoi/README.json
final_url_list = []
root_url = "https://dnazoo.s3.wasabisys.com/"
miss_list=[]
for species in species_list:
    if "/" not in species:
        species += "/"
    readme_url = root_url + species + "README.json"

    readme_local = os.path.join(output_dir, species.replace("/","") + "_readme.json")
    if not os.path.exists(readme_local) or os.path.getsize(readme_local) <= 10:
    #os.system(f"wget {readme_url} -O {readme_local}")
        download_file(readme_url, readme_local)
    if not os.path.exists(readme_local) or os.path.getsize(readme_local) <= 10:
        print(f"Error: {readme_local} does not exist or is empty")
        miss_list.append(species)
        continue
    json_info = json.load(open(readme_local,"r"))
    hic_name = json_info['links']['cMap']['name']
    species_name = hic_name.replace(".rawchrom.hic","")
    hic_url = root_url + species + species_name+"_HiC.fasta.gz"
    if "cFasta" in json_info['links']:
        
        dropbox_url = json_info['links']['cFasta']
        dropbox_url = dropbox_url.split("?")[0]
    else:
        dropbox_url = hic_url
    #final_url_list.append(hic_url)
    new_dict={species:[hic_url, dropbox_url]}
    final_url_list.append(new_dict)
import pickle
pickle.dump(final_url_list, open("../data/dnazoo_fasta_url_list.pkl","wb"))
print("Done")
print("Missed species:", miss_list)

Request fail: 404
Error: ../data/dnazoo_json/Chinchilla_x_readme.json does not exist or is empty
Request fail: 404
Error: ../data/dnazoo_json/Preliminary_readme.json does not exist or is empty
Request fail: 404
Error: ../data/dnazoo_json/annotations_readme.json does not exist or is empty
Done
Missed species: ['Chinchilla_x/', 'Preliminary/', 'annotations/']


In [ ]:
# 4 download all fasta files

import sys
import pickle
import requests
 
def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Finish download {filename}")
    else:
        print(f"Request fail: {response.status_code}")
input_pkl = "../data/dnazoo_fasta_url_list.pkl"
output_dir ="../data/multi_species_hic/"
os.makedirs(output_dir, exist_ok=True)
data = pickle.load(open(input_pkl, 'rb'))
def download_url(url,file_path):
    os.system(f"wget {url} -O {file_path}")

from multiprocessing import Pool
#you can consider use the above choice to download multiple Hi-C simultaneously to accelerate downloading speed.
for species in data:
    url_list = data[species]
    file_path = os.path.join(output_dir, f"{species}.hic")
    if isinstance(url_list, list):
        for url in url_list:
            download_url(url, file_path)
            if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
                break
    elif isinstance(url_list, str):
        download_url(url_list, file_path)
    else:
        print(f"Error: {species} url list is not valid")
print("Done")



In [ ]:
# 5 Generate the chromosme size file from the fasta file

# please first install samtools in your system: https://github.com/samtools/samtools‌
import os
import sys
input_dir = "../data/multi_species_hic/"
listfiles= [f for f in os.listdir(input_dir) if f.endswith(".fasta") or f.endswith(".fasta.gz")]
from multiprocessing import Pool
p=Pool(10)
def process_file(cur_path):
    item = os.path.basename(cur_path)
    if item.endswith(".gz"):
        fasta_path = cur_path.replace(".gz","")
        if os.path.exists(fasta_path):
            os.remove(fasta_path)
        os.system(f"gzip -d {cur_path}")
        cur_path = cur_path.replace(".gz","")
    command_line = "samtools faidx " + cur_path
    expect_output = cur_path + ".fai"
    if os.path.exists(expect_output) and os.path.getsize(expect_output) > 100:
        print(f"{cur_path} already have fai file, skip.")
        return
    os.system(command_line)

for item in listfiles:
    cur_path = os.path.join(input_dir, item)
    p.apply_async(process_file,args=(cur_path,))
p.close()
p.join()

# This is to reorganize Hi-C data into a more organized format with data stored in each chromosome
    

In [ ]:
# 6. Convert the hic files to .pkl format
# This is based on the utils/hic2array.py script
import os
import sys
input_dir = "../data/multi_species_hic/"
listfiles= [f for f in os.listdir(input_dir) if f.endswith(".hic")]
resolution = 10000
script_path= "../utils/hic2array.py"
#connvet script
for item in listfiles:
    cur_hic = os.path.join(input_dir, item)
    output_path = cur_hic.replace(".hic",".pkl")
    if os.path.exists(output_path) and os.path.getsize(output_path) > 100:
        print(f"{output_path} already exists, skip.")
        continue
    command_line= f"python3 hic2array.py {cur_hic} {output_path} {resolution} 0 2"
    #no normalization and only use the cis contact
    os.system(command_line)
print("Done")

In [ ]:
# 7. Reformat the pkl to chromosome-based pkl based on the chromosisze file

import os
import pickle
import numpy as np

def filter_region(current_data, current_start, current_end):
    from scipy.sparse import coo_matrix
    row = current_data.row
    col = current_data.col
    data = current_data.data
    select_index1 = (row>=current_start) & (row<current_end)
    select_index2 = (col>=current_start) & (col<current_end)
    select_index = select_index1 & select_index2
    new_row = row[select_index] - current_start
    new_col = col[select_index] - current_start
    new_data = data[select_index]
    new_array = coo_matrix((new_data, (new_row, new_col)), shape=(current_end-current_start, current_end-current_start))
    return new_array
def reformat_pkl(input_pkl, input_chrom_file, resolution):
    data=pickle.load(open(input_pkl, 'rb'))
    #parse the chrom size file to get start and end position
    bounding_index_info={}
    with open(input_chrom_file, 'r') as f:
        for line in f:
            info= line.strip().split("\t")
            chrom_name = info[0]
            current_size = int(np.floor(int(info[1])/resolution/2)) #the recorded fasta file is double than the actual size
            current_start = int(int(info[2])//resolution/2)
            bounding_index_info[chrom_name] = [current_start,current_size]
    final_data = {}
    for chrom in data:
        if "assembly" not in chrom:
            final_data[chrom] = data[chrom]
            continue
        print(f"Start processing {chrom}")
        print(f"Chrom size: {data[chrom].shape}")
        #read the chrom info to get the corresponding sub array
        current_data = data[chrom]
        current_max = current_data.shape[0]
        for chrom_tmp in bounding_index_info:
            current_start, current_size = bounding_index_info[chrom_tmp]
            if current_start > current_max:
                print(f"Error: {chrom_tmp} size is larger than the data size.")
                continue
            current_end = min(current_max, current_start+current_size)
            actual_size = current_end - current_start
            if actual_size<=50:#skip the small chrom
                print(f"Warning: {chrom_tmp} size is smaller than 50, skip.")
                continue
            final_data[chrom_tmp] = filter_region(current_data, current_start, current_end)
            print(f"Finish processing {chrom_tmp} in {chrom}")
            print(f"filter Chrom size: {final_data[chrom_tmp].shape}")
    return final_data
if __name__ == '__main__':
    import sys
    input_pkl_dir = "../data/multi_species_hic/"
    input_chrom_size_dir = "../data/multi_species_hic/"
    output_pkl_dir = "../data/multi_species_final_hic/"
    os.makedirs(output_pkl_dir, exist_ok=True)
    resolution = 10000
    listfiles= [f for f in os.listdir(input_pkl_dir) if f.endswith(".pkl")]
    listfiles.sort()
    for item in listfiles:
        input_pkl = os.path.join(input_pkl_dir, item)

        input_chrom_file = os.path.join(input_chrom_size_dir, item.replace(".pkl",".fasta.fai"))
        output_pkl = os.path.join(output_pkl_dir, item)
        if not os.path.exists(input_chrom_file):
            print(f"Error: {input_chrom_file} does not exist.")
            continue
        if os.path.exists(output_pkl) or os.path.exists(output_pkl.replace(".pkl","")):
            print(f"Skip: {output_pkl} already exists.")
            continue
        
        reformat_data = reformat_pkl(input_pkl, input_chrom_file, resolution)
        pickle.dump(reformat_data, open(output_pkl, 'wb'))
        print(f"Finish processing {item}")

